In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
example2_df = sqlContext.read.csv("cast_info.csv")

In [ ]:
example2_df.head(10)

In [ ]:
example2_df

In [ ]:
print (example2_df.printSchema())

In [ ]:
example2_df.registerTempTable("cast_info")

In [ ]:
example3_df = sqlContext.read.csv("movie_companies.csv")

In [ ]:
example3_df.head(10)

In [ ]:
example3_df

In [ ]:
print (example3_df.printSchema())

In [ ]:
example3_df.registerTempTable("movie_companies")

In [ ]:
example4_df = sqlContext.read.csv("title.csv")

In [ ]:
example4_df.head(10)

In [ ]:
example4_df

In [ ]:
print (example4_df.printSchema())

In [ ]:
example4_df.registerTempTable("title")

In [ ]:
#Number of companies who have produced films with love in the title

In [ ]:
%%time
from pyspark.sql.functions import count
query = """
select
    count(movie_companies._c2)
from
    movie_companies
inner join title on
    movie_companies._c1=title._c0
    
where title._c1 like '% love%'
"""
love=(sqlContext.sql(query).toPandas())

In [ ]:
love.columns = ['Count']
love

In [ ]:
example5_df = sqlContext.read.csv("movie_info.csv")

In [ ]:
example5_df.head(10)

In [ ]:
example5_df

In [ ]:
print (example5_df.printSchema())

In [ ]:
example5_df.registerTempTable("movie_info")

In [ ]:
#The titles of the 1000 more successful movies from 1990-2000

In [ ]:
%%time
from pyspark.sql.functions import count
query = """
select
    title._c1
from
    movie_info
inner join title on
    movie_info._c1=title._c0
where title._c4>=1990 and title._c4<=2000 order by (movie_info._c6-movie_info._c7) DESC limit 1000
"""
title1000=(sqlContext.sql(query).toPandas())

In [ ]:
title1000.columns = ['Movie Title']
title1000

In [ ]:
example6_df = sqlContext.read.csv("person.csv")

In [ ]:
example7_df = sqlContext.read.csv("aka_name.csv")

In [ ]:
example7_df.head(10)

In [ ]:
example6_df.head(10)

In [ ]:
example6_df

In [ ]:
example7_df

In [ ]:
example6_df.registerTempTable("person")

In [ ]:
example7_df.registerTempTable("aka_name")

In [ ]:
example8_df = sqlContext.read.csv("movie_keyword.csv")

In [ ]:
example8_df.head(10)

In [ ]:
example8_df

In [ ]:
example8_df.registerTempTable("movie_keyword")

In [ ]:
example9_df = sqlContext.read.csv("keyword.csv")

In [ ]:
example9_df.head(10)

In [ ]:
example9_df

In [ ]:
example9_df.registerTempTable("keyword")

In [ ]:
#Title of each movie that has specific keyword(design) in the plot keywords list

In [ ]:
%%time
from pyspark.sql.functions import count
query = """
select
    title._c1
from
    title
left join movie_keyword on
    title._c0=movie_keyword._c1
inner join keyword on
    movie_keyword._c2=keyword._c0
where keyword._c1 like '%design%'
"""
titlescounts=(sqlContext.sql(query).toPandas())

In [ ]:
titlescounts.columns = ['Movie Title']
titlescounts

In [ ]:
#Ratio of likes of a specific actor to those of the cast

In [ ]:
%%time
query = """
select
    aka_name._c7, cast_info._c7
from
    cast_info
inner join aka_name on
    cast_info._c1=aka_name._c1
where aka_name._c2 like '%Hanks%Tom'
"""
data_likes=sqlContext.sql(query).toPandas()

In [ ]:
data_likes

In [ ]:
data_likes.columns = ['Actor_Like', 'Cast_Like']

In [ ]:
frame1=data_likes
frame1

In [ ]:
s=data_likes.values

In [ ]:
import pandas as pd
data_likes.apply(pd.to_numeric)
s=data_likes.values
a=s.astype(int)

In [ ]:
ratio_likes=[]
for i in range(a.shape[0]):
    ratio_likes.append(a[0][0]/a[i][1])
ratio_likes   

In [ ]:
import numpy as np
df = pd.DataFrame(np.array(ratio_likes).reshape(91,1), columns = list("a"))
df.columns=['ratio likes']
df

In [ ]:
df_new = pd.concat([frame1, df],axis=1)

In [ ]:
df_new

In [ ]:
example10_df = sqlContext.read.csv("company_name.csv")

In [ ]:
example10_df.head(10)

In [ ]:
example10_df

In [ ]:
example10_df.registerTempTable("company_name")

In [ ]:
#Sum of movie gross of the movies produced by Amazon for different production years

In [ ]:
%%time
from pyspark.sql.functions import sum
query = """
select
    sum(movie_info._c6), title._c4
from
    movie_info
inner join title on
    movie_info._c1=title._c0
inner join movie_companies on
    title._c0=movie_companies._c1
inner join company_name on
    movie_companies._c2=company_name._c0
where company_name._c1='Amazon' group by (title._c4) order by (title._c4) ASC
"""
plot_movie_gross=(sqlContext.sql(query).toPandas())

In [ ]:
plot_movie_gross.columns = ['Cumulative Gross (x10^8) per Year', 'Production Year']
plot_movie_gross

In [ ]:
import matplotlib.pyplot as plt
plot_movie_gross.plot(x='Production Year', y='Cumulative Gross (x10^8) per Year',kind='bar', figsize=(12, 5),legend=False)
plt.ylabel('Cumulative Gross (x10^8) per Year')
plt.xlabel('Production Year')

In [ ]:
#Average of movie budget spent by Paramount Television for different production years

In [ ]:
%%time
query = """
select
    avg(movie_info._c7), title._c4
from
    movie_info
inner join title on
    movie_info._c1=title._c0
inner join movie_companies on
    title._c0=movie_companies._c1
inner join company_name on
    movie_companies._c2=company_name._c0
where company_name._c1='Paramount Television' group by (title._c4) order by (title._c4) ASC
"""
plot_average_budget=(sqlContext.sql(query).toPandas())

In [ ]:
plot_average_budget.columns = ['Average Budget (x10^7) per Year for the Paramount Television Company', 'Production Year']
plot_average_budget

In [ ]:
import matplotlib.pyplot as plt
plot_average_budget.plot(x='Production Year', y='Average Budget (x10^7) per Year for the Paramount Television Company',kind='bar', figsize=(12, 5),legend=False)
plt.ylabel('Average Budget (x10^7) per Year for the Paramount Television Company')
plt.xlabel('Production Year')

In [ ]:
example11_df = sqlContext.read.csv("char_name.csv")

In [ ]:
example11_df.head(10)

In [ ]:
example11_df

In [ ]:
example11_df.registerTempTable("char_name")

In [ ]:
example12_df = sqlContext.read.csv("kind_type.csv")

In [ ]:
example12_df.head(10)

In [ ]:
example12_df

In [ ]:
example12_df.registerTempTable("kind_type")

In [ ]:
example13_df = sqlContext.read.csv("role_type.csv")

In [ ]:
example13_df.head(10)

In [ ]:
example13_df.registerTempTable("role_type")

In [ ]:
#Count of times the character name "Harry" appears in all the tv series of the database

In [ ]:
%%time
query = """
select
    count(char_name._c0), title._c1
from
    char_name
inner join cast_info on
    char_name._c0=cast_info._c3
inner join movie_info on
    cast_info._c2=movie_info._c1
inner join title on
    movie_info._c1=title._c0
inner join kind_type on
    title._c3=kind_type._c0    
where char_name._c1 like '%Harry%' and kind_type._c1='tv series' group by(title._c1)
"""
count_name_appears=(sqlContext.sql(query).toPandas())

In [ ]:
count_name_appears.columns = ['Number of times a character name ("Harry") appears in all tv series', 'Tv Series Name']
count_name_appears

In [ ]:
top50=count_name_appears[0:50]
import matplotlib.pyplot as plt
top50.plot(x='Tv Series Name', y='Number of times a character name ("Harry") appears in all tv series',kind='bar', figsize=(12, 5),legend=False)
plt.ylabel('Number of times a character name ("Harry") appears in all tv series')
plt.xlabel('Tv Series Name')

In [ ]:
#Return the count of times the word keyword kill appears in the titles:

In [ ]:
%%time
query = """
select
    count(keyword._c0)
from
    keyword
where keyword._c1 like '%kill%'
"""
count_of_times=(sqlContext.sql(query).toPandas())

In [ ]:
count_of_times.columns = ['Count']
count_of_times

In [ ]:
#Return the id of keyword table where keyword is equal to "kill" or phonetic code is equal to "a2525":

In [ ]:
%%time
query = """
select
    keyword._c0
from
    keyword
where keyword._c1 like '%kill%' or keyword._c2='A2525'
"""
phonetics=(sqlContext.sql(query).toPandas())

In [ ]:
phonetics.columns = ['ID']
phonetics

In [ ]:
%%time
query = """
select
    count(keyword._c0)
from
    keyword
where keyword._c1 like '%kill%' or keyword._c2='A2525'
"""
counterids=(sqlContext.sql(query).toPandas())

In [ ]:
counterids.columns = ['Count']
counterids

In [ ]:
#Return number of tuples in table keywords where keyword field starts with kill

In [ ]:
%%time
query = """
select
    count(keyword._c0)
from
    keyword
where keyword._c1='kill%'
"""
counterkills=(sqlContext.sql(query).toPandas())

In [ ]:
counterkills.columns = ['Count']
counterkills